# Poem Classification

## Data Importation and Exploration

In [22]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kaidonkennedy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kaidonkennedy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kaidonkennedy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
train_data = pd.read_csv('Poem_classification - train_data.csv')
test_data = pd.read_csv('Poem_classification - test_data.csv')

In [3]:
train_data.head(5)

,Genre,Poem
0,Music,NaN
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...


In [4]:
test_data.head(5)

,Genre,Poem
0,Music,A woman walks by the bench I’m sitting onwith ...
1,Music,"Because I am a boy, the untouchability of beau..."
2,Music,"Because today we did not leave this world,We n..."
3,Music,"Big Bend has been here, been here. Shouldn’t i..."
4,Music,"I put shells there, along the lip of the road...."


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Genre   841 non-null    object
 1   Poem    837 non-null    object
dtypes: object(2)
memory usage: 13.3+ KB


In [6]:
train_data['Genre'].value_counts()

Genre
Music          239
Death          234
Environment    227
Affection      141
Name: count, dtype: int64

In [7]:
train_data.isnull().sum()

Genre    0
Poem     4
dtype: int64

In [8]:
#just remove the 4 null values
train_df = train_data.dropna()

In [9]:
train_df.isnull().sum()

Genre    0
Poem     0
dtype: int64

## Preprocessing

In [10]:
# clean text
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    words = nltk.word_tokenize(text)
    return words

In [11]:
#remove stop words
stop_words = set(stopwords.words('english'))

def remove_stopwords(words):
    return [word for word in words if word not in stop_words]

In [12]:
#lemmatization
lemmatizer = WordNetLemmatizer()

def lemmatize_words(words):
    return [lemmatizer.lemmatize(word) for word in words]

In [13]:
#Apply Preprocessing
def preprocess_poem(poem):
    words = clean_text(poem)
    words = remove_stopwords(words)
    words = lemmatize_words(words)
    return ' '.join(words)  # Join words back into a single string

In [14]:
train_df['cleaned_poem'] = train_df['Poem'].apply(preprocess_poem)

/var/folders/c7/symnjm6d69x6p3rtw2qvqmyc0000gn/T/ipykernel_36574/1431374668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['cleaned_poem'] = train_df['Poem'].apply(preprocess_poem)


In [15]:
# Encoding
label_encoder = LabelEncoder()
train_df['genre_encoded'] = label_encoder.fit_transform(train_df['Genre'])

# Save the mapping for later use
category_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(category_mapping)

{'Affection': 0, 'Death': 1, 'Environment': 2, 'Music': 3}


/var/folders/c7/symnjm6d69x6p3rtw2qvqmyc0000gn/T/ipykernel_36574/1316622839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['genre_encoded'] = label_encoder.fit_transform(train_df['Genre'])


In [16]:
train_df.head(1)

,Genre,Poem,cleaned_poem,genre_encoded
1,Music,In the thick brushthey spend the...,thick brushthey spend hottest part day soaking...,3


In [18]:
# split the data
X = train_df['cleaned_poem']
y = train_df['genre_encoded']

# Use train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Feature extraction

### TF-IDF

In [23]:
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Top 5000 features

# Fit and transform training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform test data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(X_train_tfidf.shape)  # Check dimensions of the output

(669, 5000)


## Modeling

### Logistic regression model